<img align="left" src="media/Assets&ArchHeader.jpg">

# Using the Db2 Console RESTful Service Class

### Db2 Console Open API
This Jupyter Notebook contains examples of how to use the resuable Python class library that encapsulates the Open APIs of the Db2 Data Management Console. Everything in the Console User Interface is available through an open and fully documented RESTful Services API. The full set of APIs are documented as part of the Db2 Data Management Console user interface. In this hands on lab you can connect to the documentation directly through this link: [Db2 Data Management Console RESTful APIs](http://localhost:11080/dbapi/api/index_enterprise.html). 

### Where to find this sample online
You can find a copy of this notebook at https://github.com/Db2-DTE-POC/db2dmc.

Let's get started by loading the db2console.ipynb class library notebook. You can see the latest copy of this notebook at...

In [ ]:
%run db2console.ipynb

### Db2 Data Management Console Connection
The first step is to create an instance of the Db2Console class (a Python object). The next cell creates an object called **databaseAPI**. The rest of this lab calls functions that are part of the Db2Console class.

The **databaseAPI** object creation requires the URL and root API of the Db2 Console RESTful API as well as the a valid userid and password that you would use to log into the console. 

**Note:** If the Db2 Data Management Console has not completed initialization, the connection below will fail. Wait for a few moments and then try it again.

In [ ]:
# Connect to the Db2 Data Management Console service
Console  = 'http://localhost:11080'
user     = 'db2inst1'
password = 'db2inst1'

# Set up the required connection
databaseAPI = Db2Console(Console+'/dbapi/v4')
if databaseAPI.authenticate(user, password) :
    print("Token Created")
else : 
    print("Token Creation Failed")
database = Console

If the connection was successfully established, the new object contains a reusable token that is used to reconnect to the console API service for each function call. You don't need to ever use the token in your code, but if you want to see what a secure token looks like run the next cell.

In [ ]:
databaseAPI.getBearerToken()

### Confirm the connection
To confirm that your connection is working you can list the Console connection profiles.

In [ ]:
databaseAPI.getConnectionProfiles()

You can also see the details for a specific connection profile.

In [ ]:
databaseAPI.getConnectionProfile('Ontime')

### Get Repository Configuration
You can also get details on the repository configuration. You can see that we are using a local database name **repo** to store all the monitoring data collected by the console. 

In [ ]:
databaseAPI.getConsoleRepository()

## Connect to a Specific Database
To use functions that work on a specific database you need to recreate the **databaseAPI** object with a specific connection profile. Different users may or may not be allowed to connect using specific profiles. So we need to re-authenticate each time we switch databases. Let's check the privleges of different users.

In [ ]:
display(databaseAPI.getUserPrivileges())

Notice that we don't have to recreate the databaseAPI object but we do need to re-authenticate. Re-authenticate against the Ontime database as db2inst1.

In [ ]:
# Connect to the Db2 Data Management Console service
user     = 'db2inst1'
password = 'db2inst1'
profile = 'Ontime'

# Authenticate to a specific connection profile
if databaseAPI.authenticate(user, password, profile) :
    print("Token Created")
else : 
    print("Token Creation Failed")
database = Console

### Catalog Functions
Now that you are connected to a specific database, in this example 'Ontime', you can call functions that let you access catalog information. You can get a list of the schemas in the Ontime database. The cell below retrieves all the rows in a dataframe and displays the first 5.

In [ ]:
databaseAPI.getSchemas().head(5)

#### Search and Count Tables and Views
You can also use capabilities that are built into the console. For example you can find out how many tables include the work "AIRCRAFT" or seach all the views (both user and catalog views) that include the word "TABLES". By default the functions below only search user tables. Adding "true" to the function call also searches the system tables. 

In [ ]:
display(databaseAPI.getSearchTableList("AIRCRAFT"))
display(databaseAPI.getSearchViewList("TABLES", "true"))

#### Tables in a Schema
The next function all returns the first five tables contained in the "ONTIME" schema

In [ ]:
databaseAPI.getTablesInSchema("ONTIME").head(5)

#### Fuzy object search
The next function returns a list of either the tables or views that match search text. You can specify the number of rows in the result set (in this example 5) and specify whether you want to search user object or user and system objects (in this example true means searching both). 

In [ ]:
display(databaseAPI.searchObjects('view',"TABLE",5,'true'))
display(databaseAPI.searchObjects('table',"AIRLINE",5,'true'))

### Running Scripts and Workloads
The Db2Console class also uses the SQL Editor service that is part of the Db2 Console to let you run Db2 scripts. Not use single SQL statments but scripts that include multiple statements. To make it easy to run scripts against different databases the fucntion requires the connection profile name, the userid and password and the sql script text. The next cell runs three SQL statements. 

It returns a JSON string that include details on each statement, its runtime, column types, the limit of returned rows, the full row count in the result set, and the actual results to the row limit. 

In [ ]:
sql = \
'''
SELECT TABSCHEMA, TABNAME, STATUS FROM SYSCAT.TABLES; 
SELECT VIEWSCHEMA, VIEWNAME, VALID FROM SYSCAT.VIEWS;
SELECT TABSCHEMA, TABNAME, COLNAME, TYPENAME, LENGTH FROM SYSCAT.COLUMNS
'''
user = 'db2inst1'
password = 'db2inst1'
profile = 'Ontime'

display(databaseAPI.runScript(profile, user, password, sql))

To make it easier to see the results, the displayResults function parses the JSON into a readable format.

In [ ]:
databaseAPI.displayResults(databaseAPI.runScript(profile, user, password, sql))

The number of cells returned is limited to 10 by default. You can add an additional parameter to the runScript command to return a much larger result set, which you can then manipulate in Python. The returnRows function converts the JSON result into a dataframe. It requires the json that is returned by runScipt and the index of the SQL result you want to work with.

In [ ]:
json = databaseAPI.runScript(profile, user, password, sql, 10000)
df = databaseAPI.returnRows(json,0)
display(df.head(5))
display(df.tail(5))

In [ ]:
df = databaseAPI.returnRows(json,1)
display(df.head(5))
display(df.tail(5))

In [ ]:
df = databaseAPI.returnRows(json,2)
display(df.head(5))
display(df.tail(5))

One of the most powerful functions built into the Db2Console class is **runWorkload**. It lets you run multiple scripts against multiple databases in a loop. This is particularly useful for demonstrating Db2 monitoring or for measuring the performance of SQL against different databases.

In the next example, two scripts are run repeatedly against all the databases currently cataloged by the Db2 Console.

The runtimes are collected along the way and returned in a dataframe.

In [ ]:
profileList = ['Ontime','Repository','Sample','Banking']
sql1 = \
'''
SELECT TABSCHEMA, TABNAME, STATUS FROM SYSCAT.TABLES; 
SELECT VIEWSCHEMA, VIEWNAME, VALID FROM SYSCAT.VIEWS;
'''
sql2 = \
'''
SELECT TABSCHEMA, TABNAME, COLNAME, TYPENAME, LENGTH FROM SYSCAT.COLUMNS
'''
user = 'db2inst1'
password = 'db2inst1'
scriptList = [sql1, sql2]
user = 'DB2INST1'
password = 'db2inst1'
profileReps = 2
scriptReps = 2
pause = 0.25

df = databaseAPI.runWorkload(profileList, scriptList, user, password, profileReps, scriptReps,pause)

display(df)

### Current Metrics Functions
Now that you can run a workload to exercise Db2, you can measure what is going on. The following function calls let you see what applications are connected to the "Ontime" database, see any statements that are currently in-flight and see the frequently used statements stored in the Db2 package cache.

The includeSystem parameter defines whether applications or statements generated by Db2 itself or the Db2 Console are included in the results.

In [ ]:
includeSystem = "true"

In [ ]:
databaseAPI.getCurrentApplicationsConnections(includeSystem)

In [ ]:
databaseAPI.getInflightCurrentList(includeSystem)

In [ ]:
databaseAPI.getCurrentPackageCacheList(includeSystem).head(5)

### Timeseries Monitoring Functions
One of the key capabilities of the Db2 Console is that is collects historical monitoring information as timeseries  data. The next set of functions returns monitoring data based on a start and endtime. 

The console and Db2 use EPOCH time, which is the number of milli-seconds since January 1st 1970. The cell below sets startTime and endTime. endTime is the current and end time. startTime is set to 12 hours earlier. 

In [ ]:
import time
from datetime import date
oneHour = 3600000
endTime = int(time.time())*1000
startTime = endTime-(oneHour*12)

#### Time Based Metrics - Summary Functions
The following functions return a total summary of the number of user statements that have run over the last 12 hours as well as the average response time in ms over that same period. 

In [ ]:
databaseAPI.getStatementsCount(startTime, endTime)

In [ ]:
databaseAPI.getAverageResponseTime(startTime, endTime)

#### Time Based Metrics - Interval Measurement Functions
The following functions return a measurement for each monitoring interval over the last 12 hours. The examples below return average response time and total rows read during each monitoring interval. The last 5 intervals are displayed

In [ ]:
databaseAPI.getResponseTime(startTime, endTime).tail(5)

In [ ]:
databaseAPI.getRowsRead(startTime, endTime).tail(5)

#### Time Based Metrics - Object Functions
The following functions return monitoring data over the last 12 hours with a summary row for each object. The latest 5 entries are displayed.

In [ ]:
databaseAPI.getTablesMetrics(startTime, endTime, includeSystem).tail(5)

In [ ]:
databaseAPI.getIndividualStatementExecution(startTime, endTime).tail(5)

In [ ]:
databaseAPI.getPackageCacheStatement(startTime, endTime, includeSystem).tail(5)

## What's Next
Try creating your own workload in a seperate notebook that takes at least an hour to run. You can even use the workload example above and increase the number of repetitions. In a seperate notebook, try running the monitoring functions or use the Db2 Console to see how Db2 responds under load.  

#### Credits: IBM 2019-2021, Peter Kohlmann [kohlmann@ca.ibm.com]